In [ ]:
import torch

def optimize_portfolio(covariance, expected_return, risk_coefficient=1, number_of_bits=3, agents=10):
    """
    Optimize portfolio allocation using simplified simulated bifurcation.
    
    Args:
        covariance: Asset covariance matrix
        expected_return: Expected returns vector
        risk_coefficient: Risk aversion parameter (higher = more conservative)
        number_of_bits: Number of bits to represent each asset quantity
        agents: Number of parallel optimization attempts
        
    Returns:
        portfolio: Optimized portfolio weights
        gain: Expected gain of the optimized portfolio
    """
        
    # Get dimensions
    assets = covariance.shape[0]
    
    # Prepare quadratic and linear coefficients
    quadratic = -0.5 * risk_coefficient * covariance
    linear = expected_return
    
    # Initialize particles (position and momentum)
    position = 2 * torch.rand(assets, agents) - 1
    momentum = 2 * torch.rand(assets, agents) - 1
    
    # Run simplified simulated bifurcation
    time_step = 0.05
    scaling = 0.5 * (assets - 1)**0.5 / torch.sqrt(torch.sum(quadratic**2))
    
    # Main optimization loop
    for step in range(1000):
        # Calculate pressure (annealing parameter)
        pressure = min(time_step * step * 0.01, 1.0)
        
        # Update momentum and position
        momentum += time_step * (pressure - 1.0) * position  
        position += time_step * momentum
        momentum += time_step * scaling * quadratic @ position
        
        # Apply boundary conditions
        momentum[torch.abs(position) > 1.0] = 0.0
        position = torch.clamp(position, -1.0, 1.0)
    
    # Get final spins and convert to integers
    spins = torch.where(position >= 0.0, 1.0, -1.0)
    max_val = 2**number_of_bits - 1
    weights = ((spins + 1) / 2) * max_val
    
    # Calculate objective for all agents
    gains = torch.zeros(agents)
    for i in range(agents):
        w = weights[:, i]
        gains[i] = w @ expected_return - 0.5 * risk_coefficient * w @ covariance @ w
    
    # Return best solution
    best_idx = torch.argmax(gains)
    portfolio = weights[:, best_idx]
    gain = gains[best_idx].item()
    
    return portfolio, gain


# Example usage
if __name__ == "__main__":
    # Test data
    covariance = torch.tensor([[1.0, 1.2, 0.7], 
                              [1.2, 1.0, -1.9], 
                              [0.7, -1.9, 1.0]])
    expected_return = torch.tensor([0.2, 0.05, 0.17])
    
    # Run optimization
    portfolio, gain = optimize_portfolio(
        covariance,
        expected_return,
        risk_coefficient=1,
        number_of_bits=3,
        agents=10
    )
    
    print(f"Optimized portfolio: {portfolio}")
    print(f"Expected gain: {gain:.4f}")

Optimized portfolio: tensor([0., 7., 7.])
Expected gain: 45.6400


In [2]:

assert torch.equal(torch.tensor([0.0, 7.0, 7.0]), portfolio)
